---------------------------------------------------------------------------------------------------------------------------
Este tutorial se apoya en el modelo de ejemplo que provee `dinsar`, el cual está construido a partir de conjuntos de datos reales públicos de una pequeña zona cercana al Parque Nacional de Doñana. Se puede acceder al modelo a través de `dinsar.example.get_model()` y a las rutas relativas a los datos, mediante `dinsar.example.get_path()`.
Para más información sobre el origen de los datos, consulta el fichero *Data Sources.txt*, albergado en la misma carpeta que este tutorial.

Para **más información** sobre el uso de cualquier función, método, clase o módulo, utiliza el método `help` para acceder a su documentación:

`>>> help(dinsar.parts)` o bien presione la tecla del **tabulador** en Jupyter Notebook para acceder a los métodos de un objeto.

---------------------------------------------------------------------------------------------------------------------------

## 1. Model creation
Las funcionalidades del programa giran en torno a un **modelo** de datos, el cual incluirá todos los conjuntos de datos DInSAR y bases de datos que se consultar en la zona de estudio, así como permitirá acceder a la mayor parte de las funciones del programa.

Este modelo se crea a través de la clase `Model` y, puesto que la manera más sencilla de analizar los puntos de deformación (persistent or permanent scatterers; PS) es mediante su **agregación en polígonos**, la construcción de esta clase requiere de un archivo espacial de tipo poligonal, introducido como la ruta relativa a su ubicación. De esta manera, cada uno de estos polígonos definirá las distintas áreas de análisis del modelo, también conocidas como [ADA's](https://open.igme.es/xmlui/bitstream/handle/20.500.12468/708/fast_detection_ground_2017.pdf?sequence=1) (Active Deformation Areas), en adelante llamadas como *agregados*.

Aunque la correcta generación de estos agregados requiere de la aplicación de técnicas específicas de agregación de puntos, una forma sencilla de generar esta capa de polígonos es a través de la herramienta 'Aggregate Points', de QGIS

In [1]:
import dinsar

fname = dinsar.example.get_path('agregados')
Doñana = dinsar.Model(fname)

## 2. Definition of Model's parts

Una vez definido el Modelo de la zona, la siguiente operación a realizar consiste en la definición de las **partes** que lo componene. Existen cuatro tipos posibles de partes, siendo cada una de ellas un objeto distinto, y encontrándose intensamente explicadas en su documentación. Ejemplo: ver `Dataset?`

    . Dataset
    . Piezometria
    . Precipitacion
    . DataBase
    
Mientras que la clase `Dataset` alberga un conjunto de datos DInSAR, el resto de clases representan base de datos, las cuales pueden tener información espacial asociado o no. Entre estas últimas, `Piezometria` y `Precipitacion` están diseñadas especialmente para albergar bases de datos piezométricas y pluviométricas, respectivamente, introduciéndose datos de cualquier otra naturaleza a través de la clase `DataBase`.

Se pueden definir y añadir al modelo tantas partes como se desee, requieriendo la construcción de cada una de ellas la ruta de acceso a la ubicación del archivo, así como el nombre con el que se referirán, el color con el que se visualizrá su contenido u otras opciones (ver **kwargs de cada función).

### DInSAR datasets definition:   

In [3]:
# Defining Ascending Track (called 'Asc') of Sentinel DInSAR dataset.
fname_1 = dinsar.example.get_path('sentinel-asc')
Asc = dinsar.Dataset(fname_1, name='Asc', color='#99F550')

# Defining Descending Track (alias, 'Desc') of Sentinel DInSAR dataset.
fname_2 = dinsar.example.get_path('sentinel-desc')
Desc = dinsar.Dataset(fname_2, name='Desc', color='#FBA608')

Las partes que no son `Dataset` pueden tener **información espacial** asociada o no. Ésta se introduce a través del método `Part.append_geometry`, y su correcto funcionamiento requiere que se traten a los sensores exactamente con el mismo nombre que vincula en la base de datos.

###  Piezometry and Rainfall databases

In [ ]:
# Defining piezometry database (called 'Piezo_bbdd'), with other arguments ('sep')
# passed in this case to pd.read_file function.
fname = dinsar.example.get_path('piezometria_bd')
bbdd = dinsar.Piezometria(fname, alias='Piezo_bbdd', sep='\t')

# Adding spatial data to 'Piezo_bbdd'
shp_fname = dinsar.example.get_path('piezometria_shp')
bbdd.append_geometry(shp_fname)

#***************************************************************************************************************************
fname = dinsar.example.get_path('precipitacion_bd')
precipi = dinsar.Precipitacion(fname, alias='P', sep='\t')

shp_precipi = dinsar.example.get_path('precipitacion_shp')
precipi.append_geometry(shp_precipi)

In [ ]:
## 3. Adding parts to Model

In [ ]:
Doñana.append(Asc)      # Ascending 2014-2020
Doñana.append(Desc)     # Descending 2017-2020
Doñana.append(bbdd) # Piezometría (bbdd_wavelets)
Doñana.append(precipi)  # Precipitación (Junta de Andalucía)

## Otra manera rápida de definición del modelo de ejemplo (nuevo)

In [ ]:
Doñana = dinsar.example.get_model()

## Funcionalidades varias:

In [ ]:
Doñana.study(2).plot() # Plotear las series temporales del agregado 2

In [ ]:
Doñana.study(2).mapa() # Representar espacialmente el agregado 2 y sus PS.

In [ ]:
 # Cojo los cinco primeros PS del Dataset 'Ascending' (dessde fuera del modelo ) y calculo su deformación promedio
ps = Asc.ps[0:5] 
Asc.subset(ps).plot(plot_average=True)

In [ ]:
# Cojo el primer piezómetro de la base de datos de piezometría y ploteo su serie temporal
piezo = bbdd.piezos[0]
bbdd.plot(piezo)

# A través del método 'take' accedo al array con su serie tempral: --> bbdd.take(piezo)

In [ ]:
# Selecciono la estación de Almonte y ploteo su serie temporal según su desviación acumulada
# Puedo ver qué estaciones hay en la base de datos así: --> precipi.estaciones
precipi.plot('Almonte', values='dsv')

In [ ]:
precipi.estaciones